# 予測結果を可視化・提出

・horizon=16の結果

・S3のパス  
s3://work-kaggle-ohio/store-sales-time-series-forecasting/export_targetonly_noholiday_16days/


In [1]:
!pip install botocore==1.22.5

/opt/conda/lib/python3.7/site-packages/secretstorage/dhcrypto.py:16: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
/opt/conda/lib/python3.7/site-packages/secretstorage/util.py:25: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes


## エラー対応
pandasでS3パス指定でファイルを読み込むには、botocoreのバージョンが1.22.5以下である必要がある。

https://stackoverflow.com/questions/69994834/attributeerror-aioclientcreator-object-has-no-attribute-register-lazy-block

In [2]:
import boto3
import botocore

In [3]:
boto3.__version__

'1.20.23'

In [4]:
botocore.__version__

'1.22.5'

In [5]:
import pandas as pd

In [6]:
df = pd.DataFrame()
for i in range(200):
    #print(i)
    df_tmp = pd.read_csv(f's3://work-kaggle-ohio/store-sales-time-series-forecasting/export_forecast_tgtonly_log1p_holidays/export_forecast_tgtonly_log1p_holidays_2022-01-21T03-59-51Z_part{i}.csv')
    df = pd.concat([df, df_tmp])

In [7]:
df

,item_id,date,store_nbr,p10,p50,p90
0,automotive,2017-08-16T00:00:00Z,34,0.616184,1.399378,2.093367
1,automotive,2017-08-17T00:00:00Z,34,0.561647,1.321082,2.150555
2,automotive,2017-08-18T00:00:00Z,34,0.602325,1.379759,2.187330
3,automotive,2017-08-19T00:00:00Z,34,0.680245,1.465421,2.248687
4,automotive,2017-08-20T00:00:00Z,34,0.694800,1.469516,2.262806
...,...,...,...,...,...,...
139,prepared foods,2017-08-27T00:00:00Z,50,4.168710,4.537978,4.901857
140,prepared foods,2017-08-28T00:00:00Z,50,3.965215,4.352189,4.759768
141,prepared foods,2017-08-29T00:00:00Z,50,3.840932,4.246447,4.627309
142,prepared foods,2017-08-30T00:00:00Z,50,3.947527,4.329600,4.728683


In [8]:
import numpy as np
df['family'] = df['item_id'].str.upper()
df['date'] = df['date'].str[0:10]
df['sales'] = np.expm1(df['p50'])

In [9]:
df

,item_id,date,store_nbr,p10,p50,p90,family,sales
0,automotive,2017-08-16,34,0.616184,1.399378,2.093367,AUTOMOTIVE,3.052678
1,automotive,2017-08-17,34,0.561647,1.321082,2.150555,AUTOMOTIVE,2.747475
2,automotive,2017-08-18,34,0.602325,1.379759,2.187330,AUTOMOTIVE,2.973943
3,automotive,2017-08-19,34,0.680245,1.465421,2.248687,AUTOMOTIVE,3.329364
4,automotive,2017-08-20,34,0.694800,1.469516,2.262806,AUTOMOTIVE,3.347131
...,...,...,...,...,...,...,...,...
139,prepared foods,2017-08-27,50,4.168710,4.537978,4.901857,PREPARED FOODS,92.501530
140,prepared foods,2017-08-28,50,3.965215,4.352189,4.759768,PREPARED FOODS,76.648278
141,prepared foods,2017-08-29,50,3.840932,4.246447,4.627309,PREPARED FOODS,68.856745
142,prepared foods,2017-08-30,50,3.947527,4.329600,4.728683,PREPARED FOODS,74.913899


In [10]:
df_test = pd.read_csv('../input/test.csv')

In [11]:
df_test

,id,date,store_nbr,family,onpromotion
0,3000888,2017-08-16,1,AUTOMOTIVE,0
1,3000889,2017-08-16,1,BABY CARE,0
2,3000890,2017-08-16,1,BEAUTY,2
3,3000891,2017-08-16,1,BEVERAGES,20
4,3000892,2017-08-16,1,BOOKS,0
...,...,...,...,...,...
28507,3029395,2017-08-31,9,POULTRY,1
28508,3029396,2017-08-31,9,PREPARED FOODS,0
28509,3029397,2017-08-31,9,PRODUCE,1
28510,3029398,2017-08-31,9,SCHOOL AND OFFICE SUPPLIES,9


## 結合方針
・キーは、family, data, store_nbr。予測値（sales）はp50を使う。  
・その後、id, salesのみ切り出せば、提出できる。

In [12]:
df_mrg = pd.merge(df_test, df, how='left', on=['date', 'store_nbr', 'family'])

In [13]:
df_mrg[['id' ,'sales']].to_csv('../submit/sub_forecast_16days_only_target_log1p_holidays.csv', index=False)

# 完